# Step 5: Web UI (Chatting with Open LLM)

本ステップでは、Step 2以降で経験してきたナレッジを活用して簡易的なRAGアプリケーションの構築を経験します。
- Step 2以降で準備したRetrieverとGeneratorを使ってRAGの実装を用意する
- RAGの実装を活用しやすいように、Gradioを用いてFront-endになるWeb UIを実装する
- Web UIから質問を投げかけてRAGを体験する
![Step5](../image/rag-overview-step5.png)

## 0. 事前準備

### 共通処理/定数定義
全ステップで共通して使用する定数とバナークラスを読み込みます。

In [ ]:
from mylib import myconstant
from mylib.MyBanner import MyBanner

### Access Token入力
Open LLMの取得に必要となるHugging FaceのAccess Tokenを入力します。（事前に発行して入手しておきます）

In [ ]:
MyBanner.start()

from getpass import getpass
HF_ACCESS_TOKEN = getpass("Hugging Face の Access Token を入力して Enter Key を押してください: ")

MyBanner.finish()

### パッケージインストール
本ステップの処理で依存するパッケージをインストールします。

In [ ]:
MyBanner.start()

!python -V
!pip install langchain
!pip install langchain-huggingface
!pip install langchain_milvus
!pip install accelerate
!pip install --upgrade gradio

!pip install ipywidgets
!pip install urllib3==1.26.20

MyBanner.finish()

### import
本ステップの処理で依存するモジュールを読み込みます。

In [ ]:
MyBanner.start()

from mylib.MyEmbedding import MyEmbedding
from mylib.MyMilvus import MyMilvus
from mylib.MyOpenLlmList import MyOpenLlmList
from mylib.MyGenerator import MyGenerator
import gradio as gr

MyBanner.finish()

## 1. 生成: VectorDB

### 【準備】Embedding Model読込
Embedding Modelをメモリに読み込みます。

In [ ]:
MyBanner.start()

embeddings = MyEmbedding.get_model()
display(embeddings)

MyBanner.finish()

### 【準備】Vector DB接続
エンベディングで利用するEmbedding Modelと接続情報を渡してベクトルデータベースへ接続します。
- 接続失敗した場合は、Milvus(milvus-standalone コンテナ)が起動しているか確認します

In [ ]:
MyBanner.start()

# connect to VectorDB
vector_db = MyMilvus(
    myconstant.VDB_HOST, myconstant.VDB_PORT,
    myconstant.VDB_USER, myconstant.VDB_PASS, embeddings)
print(f"{vector_db=}")

MyBanner.finish()

## 2. 生成: Generator

### 【生成】Open LLM一覧
取り扱うOpen LLM名の一覧を管理するオブジェクトを生成します。

In [ ]:
MyBanner.start()

openllm_list = MyOpenLlmList([2, 4])

MyBanner.finish()

### 【生成】Generator Object
プロンプトを使ってLLMへ回答案の作成依頼を連携するGeneratorオプジェクトを生成します。

In [ ]:
MyBanner.start()

generator = MyGenerator(openllm_list.getAll(), HF_ACCESS_TOKEN)
display(generator)

MyBanner.finish()

## 3. Web UI to chat with LLM

### Script for Web UI written in Gradio
- Components manual in Gradio: https://www.gradio.app/docs/gradio/interface

以下の画面で構成されるWeb UIを使ってRAGを体験します。

| ![Step5](../image/chatting_with_openllm.png) |
|:---:|
| 図. Web UIの画面構成 |

次のセルはWeb UIを構築するソースコードで、実行後にソースコードのセル直下に表示されるURLでWeb UIにアクセスします。

In [ ]:

#--------------------------------------------------
# define global variables
#--------------------------------------------------
gbal_response = None
gbal_collections = vector_db.get_collections()
print(f"{gbal_collections=}")

#--------------------------------------------------
# define event procedures and sub functions
#--------------------------------------------------

def create_chain(drp_llm, is_conversation, 
                 is_retrieval, drp_collection,
                 is_apply_cond, nmb_rec, sld_sim, 
                 is_edit_template, txt_template):

    # decide a template for prompt
    template = None
    if is_edit_template == "On":
        template = txt_template
    elif (not gbal_response is None) and (is_conversation == "On"):
        template = generator.make_template_for_conversation(drp_llm, gbal_response)
    # create a chain    
    if is_retrieval == "On":
        docstore = vector_db.connect(drp_collection)
        if is_apply_cond == "On":
            retriever = vector_db.get_retriever(docstore, nmb_rec, sld_sim)
        else:
            retriever = vector_db.get_retriever(docstore)
        chain = generator.create_chain(retriever, drp_llm, template)
    else:
        chain = generator.create_chain_not_retriever(drp_llm, template)
    return chain

def chatbot_response(message, history, 
                     drp_llm, is_conversation,
                     is_retrieval, drp_collection,
                     is_apply_cond, nmb_rec, sld_sim,
                     is_edit_template, txt_template):
    MyBanner.start()
    global gbal_response
    
    print(f"{message=}",
          f"{drp_llm=}", f"{is_conversation=}",
          f"{is_retrieval=}", f"{drp_collection=}",
          f"{is_apply_cond=}", f"{nmb_rec=}", f"{sld_sim=}")

    # Create a chain
    chain = create_chain(drp_llm, is_conversation, 
                         is_retrieval, drp_collection,
                         is_apply_cond, nmb_rec, sld_sim, 
                         is_edit_template, txt_template)
    
    response = chain.invoke(message)
    gbal_response = response
    print(f"{response=}" + "\n" + ("-" * 30))
    
    answer = generator.extract_answer_from_response(drp_llm, response)
    print(f"{answer=}" + "\n" + ("-" * 30))
    
    MyBanner.finish()
    print()
    return answer

def change_radio_similarity(sld_sim):
    print(sld_sim)
    return sld_sim

def change_dropdown_llm():
    return gr.update(value="Off"), gr.update(value="", visible=False)

def change_radio_retrieval(is_retrieval):
    if is_retrieval == "On":
        return gr.update(visible=True)
    else:
        return gr.update(visible=False)

def change_radio_apply_cond(is_apply_cond):
    flg_visible = False
    if is_apply_cond == "On":
        flg_visible = True
    return gr.update(visible=flg_visible), gr.update(visible=flg_visible)

def change_radio_edit_template(is_edit_template, txt_template, drp_llm):
    if is_edit_template == "On":
        template = txt_template
        if template.strip() == "":
            template = generator.get_template(drp_llm)
        return gr.update(visible=True, value=template)
    else:
        return gr.update(visible=False)

#--------------------------------------------------
# draw a gui
#--------------------------------------------------
# with gr.Blocks(fill_height=True, theme=gr.themes.Citrus()) as demo:
with gr.Blocks(fill_height=True) as demo:
    gr.Markdown("## Chatting with Open LLM (App for understanding RAG mechanism)")
    with gr.Accordion("Use conditions:", open=True):
        with gr.Tab("Basic Conditions"):
            with gr.Row():
                drp_llm = gr.Dropdown(openllm_list.getAll(),
                                      label="Open LLM:",
                                      info="Select a LLM to generate an answer.",
                                      value=openllm_list.get(0), interactive = True, visible=True)
                is_conversation = gr.Radio(choices=["On", "Off"], type="value",
                                           label="Conversation:",
                                           info="Respond with a continuous conversation..",
                                           value="Off", interactive=True)
                is_retrieval = gr.Radio(choices=["On", "Off"], type="value",
                                        label="Retriever(Similarity vector search):",
                                        info="Provide with data for similarity search to answer.",
                                        value="On", interactive=True)
                drp_collection = gr.Dropdown(gbal_collections,
                                             label="Collection:",
                                             info="Select a collection for similarity search.",
                                             value=gbal_collections[0],
                                             interactive = True, visible=True)
        with gr.Tab("Retriever Condition") as tab_retriever:
            with gr.Row():
                is_apply_cond = gr.Radio(choices=["On", "Off"], type="value", value="Off", 
                                         label="Detail conditions for retriever:",
                                         info="Choose a mode to adjust using the conditions on the right. If this feature is enabled, there will be no continuous conversation.",
                                         interactive=True)
                nmb_rec = gr.Number(value=15,
                                    label="1. Number of records:",
                                    info="Input a value as 'k' of similarity_search().",
                                    visible=False, interactive=True)
                sld_sim = gr.Slider(minimum=0.6, maximum=1.0, step=0.01, value=0.7,\
                                    label="2. Similarity:",
                                    info="Choose a lower threshold between 0.6 and 1.0.", 
                                    visible=False, interactive=True)
        with gr.Tab("Editing a template for prompt"):
            with gr.Row():
                with gr.Column(scale=1):
                    is_edit_template = gr.Radio(choices=["On", "Off"], type="value",
                                                label="Template for prompt:",
                                                info="Choose a mode to edit a template for prompt. If this feature is enabled, there will be no continuous conversation.",
                                                value="Off", interactive=True)
                with gr.Column(scale=3):
                    txt_template = gr.TextArea(label="The editing area:",
                                               info="Edit to fine-tune a template to be inputted into LLM.",
                                               visible=False)

    # attach a component and an event procedure
    drp_llm.change(fn=change_dropdown_llm, outputs=[is_edit_template, txt_template])
    is_retrieval.change(fn=change_radio_retrieval, inputs=[is_retrieval], outputs=[tab_retriever])
    is_apply_cond.change(fn=change_radio_apply_cond, inputs=[is_apply_cond], outputs=[sld_sim, nmb_rec])
    is_edit_template.change(fn=change_radio_edit_template, inputs=[is_edit_template, txt_template, drp_llm], outputs=[txt_template])

    # placed a chatbox!
    cht_bot = gr.ChatInterface(fn=chatbot_response, type="messages",\
                               additional_inputs=[
                                   drp_llm,
                                   is_conversation,
                                   is_retrieval,
                                   drp_collection,
                                   is_apply_cond,
                                   nmb_rec, sld_sim,
                                   is_edit_template,
                                   txt_template,
                               ],
                               fill_height=True)


demo.launch()

## 4. Wrap up

以上をもって全てのステップが終了となります。一連のステップを通じてLangChainを用いたRAGの実装を体験することができたと思います。